# 🌳 VoxCity View Index Analysis

Analyze **Green View Index (GVI)** and **Sky View Index (SVI)** using 3D ray-casting through your voxel city model.

## What are View Indices?

| Index | Description | Use Case |
|-------|-------------|----------|
| **GVI** | Percentage of view occupied by vegetation | Urban greenery assessment |
| **SVI** | Percentage of visible sky | Urban canyon analysis, thermal comfort |

## Key Features

- Ray-casting from observer viewpoint through 3D voxel model
- Configurable azimuth and elevation sampling
- Tree transmissivity modeling (light passes through tree canopy)
- Export results as colored OBJ meshes

## Prerequisites

```python
pip install voxcity
```

In [ ]:
# %pip install voxcity

from voxcity.generator import get_voxcity
from voxcity.geoprocessor.draw import draw_rectangle_map_cityname
from voxcity.simulator.view import get_view_index

cityname = "Tokyo, Japan"
meshsize = 5

# m, rectangle_vertices = draw_rectangle_map_cityname(cityname, zoom=15)
# m

rectangle_vertices = [
    (139.760, 35.680),  # SW
    (139.760, 35.690),  # NW
    (139.770, 35.690),  # NE
    (139.770, 35.680)   # SE
]

city = get_voxcity(
    rectangle_vertices,
    meshsize=meshsize,
    building_source='OpenStreetMap',
    land_cover_source='OpenStreetMap',
    canopy_height_source='High Resolution 1m Global Canopy Height Maps',
    dem_source='DeltaDTM',
    output_dir='output/view_demo'
)

city.voxels.classes.shape


---
## 🌳 Green View Index (GVI)

Calculate the percentage of visible vegetation from each grid cell.

### Advanced Parameters

| Parameter | Description | Default |
|-----------|-------------|---------|
| `N_azimuth` | Number of azimuth angles | 72 |
| `N_elevation` | Number of elevation angles | 12 |
| `elevation_min_degrees` | Minimum elevation angle | -15° |
| `elevation_max_degrees` | Maximum elevation angle | 30° |
| `ray_sampling` | Sampling method: 'grid' or 'fibonacci' | 'fibonacci' |
| `tree_k` | Extinction coefficient for tree canopy | 0.5 |
| `tree_lad` | Leaf Area Density (LAD) | 1.0 |

In [ ]:
gvi_kwargs = {
    "view_point_height": 1.5,
    "colormap": "Greens",
    "obj_export": True,
    "output_directory": "output/view_demo",
    "output_file_name": "gvi",
    "alpha": 1.0,
    # Ray sampling controls
    "N_azimuth": 72,
    "N_elevation": 12,
    "elevation_min_degrees": -15,
    "elevation_max_degrees": 30,
    "ray_sampling": "fibonacci",  # 'grid' or 'fibonacci'
    # Tree transmittance controls
    "tree_k": 0.5,
    "tree_lad": 1.0,
}

gvi_grid = get_view_index(city, mode='green', **gvi_kwargs)

gvi_grid.shape


---
## 🌤️ Sky View Index (SVI)

Calculate the percentage of visible sky from each grid cell.

For SVI, the elevation range is typically from horizon (0°) to zenith (90°).

In [ ]:
svi_kwargs = gvi_kwargs.copy()
svi_kwargs["colormap"] = "BuPu_r"
svi_kwargs["output_file_name"] = "svi"
svi_kwargs["elevation_min_degrees"] = 0
svi_kwargs["elevation_max_degrees"] = 90

svi_grid = get_view_index(city, mode='sky', **svi_kwargs)

svi_grid.shape


---
## 📊 Interpreting Results

- **GVI values** range from 0 to 1 (0% to 100% vegetation visibility)
- **SVI values** range from 0 to 1 (0% to 100% sky visibility)
- Results are exported as colored OBJ files for visualization in Blender/Rhino
- Higher `N_azimuth` and `N_elevation` values give more accurate but slower results

## Next Steps

- `demo_solar.ipynb` - Solar irradiance simulation
- `demo_landmark.ipynb` - Landmark visibility analysis
- `demo_3d_visualization.ipynb` - Advanced 3D visualization